In [10]:
import pathlib
from pathlib import Path
import os
import re

import pandas as pd
import numpy as np

import tensorflow as tf
import keras
from keras import layers, Model, optimizers
import librosa


import scipy
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
sample_rate = 16000
print(type(sample_rate))

<class 'int'>


In [9]:
# mention absolute import path w.r.t the current directory

base_dir = Path.cwd().parent.parent
base_dir

PosixPath('/Users/AnanyaPal1/Documents/TUD/5th semester/resampling and simulations/anomaly-detection-in-sound-data')

In [11]:
# 3) Parameter (alle als Integer, wo nötig)

sample_rate = 16000
n_fft = 1024
hop_length = 512
n_mels = 128

P = 2                       # Kontext links/rechts
n_context = 2 * P + 1          # 5 Frames
input_dim = n_mels * n_context    # 128 * 5 = 640

train_dir = base_dir / "data" / "fan" / "train"
test_dir  = base_dir / "data" / "fan" / "test"

epochs_ae = 100
batch_size_ae = 512

In [12]:
train_dir

PosixPath('/Users/AnanyaPal1/Documents/TUD/5th semester/resampling and simulations/anomaly-detection-in-sound-data/data/fan/train')

Compute context stacked log-mel features for the audio files.

In [13]:
def compute_logmel_context(file_path,
                           sample_rate=16000,
                           n_fft=1024,
                           hop_length=512,
                           n_mels=128,
                           P=2):
    """
    Compute context-stacked log-mel features for a single audio file.

    Returns
    -------
    out : np.ndarray of shape (T_eff, n_mels * (2P+1))
        Each row is a flattened (n_mels x (2P+1)) log-mel context window.
        Returns None if the file is too short to form one full context window.
    """
    n_context = 2 * P + 1
    input_dim = n_mels * n_context

    # 1) Load waveform (mono) at target sample rate
    y, sr = librosa.load(file_path, sr=sample_rate, mono=True)

    # 2) Mel spectrogram (power)
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels
    )

    # 3) Convert power -> dB (log scale), reference to max (like ref=np.max)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Ensure shape is (n_mels, T)
    M = np.asarray(mel_db)
    if M.ndim != 2:
        M = M.reshape(n_mels, -1)

    n_frames = M.shape[1]

    # 4) Discard too-short files
    if n_frames < n_context:
        return None

    # 5) Build context-stacked features
    T_eff = n_frames - 2 * P
    out = np.zeros((T_eff, input_dim), dtype=np.float32)

    idx = 0
    for t in range(P, n_frames - P):  # 0-based: centers P .. n_frames-P-1
        win = M[:, (t - P):(t + P + 1)]      # shape: (n_mels, n_context)
        out[idx, :] = win.reshape(-1, order="F")  # match R as.numeric() column-major flatten
        idx += 1

    return out


Load Data and ID

In [14]:
train_files = list(train_dir.rglob("*.wav"))
test_files  = list(test_dir.rglob("*.wav"))

display(f"number of train files:{len(train_files)}")
display(f"number of test files: {len(test_files)}")

'number of train files:3675'

'number of test files: 1875'

In [15]:
type(train_files)

list

In [16]:
def get_id(x):
    return re.sub(r".*id_(..)_.*", r"\1", os.path.basename(x))

train_ids  =  [get_id(p) for p in train_files]
test_ids   = [get_id(p) for p in test_files]
unique_ids = sorted(set(train_ids))

Implement an AE.


Remark: Settings specific to the structure of the AE (hyperparameters, structure) are taken from the benchmark AE method from the following research paper.


In [17]:
def build_model(input_dim):

    input = layers.Input(shape=(input_dim,))

    # Encoder

    x = layers.Dense(128)(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    # Bottleneck 8D
    x = layers.Dense(8)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    # Decoder
    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    output = layers.Dense(input_dim)(x)

    model = keras.Model(input, output)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss = "mse"
    )

    return model


In [18]:
print(type(train_ids), np.shape(train_ids))
print(type(test_ids), np.shape(test_ids))
print(train_ids)
print(test_ids)


<class 'list'> (3675,)
<class 'list'> (1875,)
['04', '00', '06', '00', '06', '02', '04', '04', '06', '02', '04', '04', '02', '00', '00', '04', '02', '02', '04', '06', '06', '00', '04', '00', '06', '00', '00', '06', '04', '06', '00', '00', '06', '02', '04', '06', '00', '06', '00', '02', '04', '04', '02', '04', '02', '02', '04', '06', '00', '00', '06', '00', '06', '04', '02', '06', '00', '00', '06', '06', '00', '06', '00', '04', '02', '04', '02', '02', '04', '02', '04', '04', '02', '00', '06', '06', '00', '06', '00', '00', '06', '06', '00', '04', '02', '06', '00', '06', '02', '02', '00', '06', '06', '04', '02', '02', '04', '00', '02', '04', '04', '02', '06', '00', '06', '04', '02', '06', '00', '06', '00', '02', '02', '04', '00', '06', '06', '00', '02', '02', '00', '06', '04', '02', '02', '00', '06', '06', '00', '04', '06', '00', '00', '06', '02', '04', '02', '06', '02', '04', '02', '04', '06', '00', '04', '02', '02', '06', '00', '06', '00', '06', '02', '04', '02', '00', '06', '00', '06',

Precompute Features ONCE (cache), then 100 Bootstrap runs total

In [21]:
# feature cache (compute once per file)

print("\nPrecompute train features(once)...\n")
train_feat_all = {f: compute_logmel_context(f) for f in tqdm(train_files)}

print("\nPrecompute test features(once)...")
test_feat_all = {f: compute_logmel_context(f) for f in tqdm(test_files)}

# drop NULL files globally
valid_train = [f for f, feat in train_feat_all.items() if feat is not None]
valid_test = [f for f, feat in test_feat_all.items() if feat is not None]

# filter file lists
train_files_v = valid_train
test_files_v = valid_test

# filter IDs using the same validity rule 
train_file_to_id = dict(zip(train_files, train_ids))
test_file_to_id = dict(zip(test_files, test_ids))

train_ids_v = [train_file_to_id[f] for f in train_files_v]
test_ids_v = [test_file_to_id[f] for f in test_files_v]

# filter feature caches to valid entries only
train_feat_all = {f: train_feat_all[f] for f in train_files_v}
test_feat_all = {f: test_feat_all[f] for f in test_files_v}

# unique sorted training IDs
unique_ids = sorted(set(train_ids_v))


Precompute train features(once)...



100%|██████████| 3675/3675 [00:24<00:00, 152.16it/s]



Precompute test features(once)...


100%|██████████| 1875/1875 [00:12<00:00, 155.82it/s]


Bootstrap settings (100 runs total)

In [22]:
R = 100
np.random.seed(123)

Helper: training+evaluation for one ID given bootstrapped train file list

In [25]:
def run_one_id_if(id, 
                  boot_train_files_for_id, 
                  max_frames_per_clip=None,
                  epochs=20, 
                  batch_size=256, 
                  verbose=0):
    """
    Bootstrap run for one ID using an autoencoder for anomaly detection.
    """
    train_feat_list = [train_feat_all[f] for f in boot_train_files_for_id if f in train_feat_all] 
    if len(train_feat_all)==0:
        return None
    
    # ---------------------------------------------------
    # Optional speed-up: cap frames per train clip
    # ---------------------------------------------------
    if max_frames_per_clip is not None:
        max_frames_per_clip = int(max_frames_per_clip)
        new_train_feat_list = []

        for M in train_feat_list:
            if M is None or M.shape[0] == 0:
                continue

            if M.shape[0] <= max_frames_per_clip:
                new_train_feat_list.append(M)
            else:
                idx = np.random.choice(
                    M.shape[0], max_frames_per_clip, replace=False
                )
                new_train_feat_list.append(M[idx, :])

        train_feat_list = new_train_feat_list

        if len(train_feat_list) == 0:
            return None

    # ---------------------------------------------------
    # Stack training frames
    # ---------------------------------------------------
    X_train = np.vstack(train_feat_list)

    if X_train.shape[0] < 2:
        return None

    input_dim = X_train.shape[1]

    # ---------------------------------------------------
    # Fixed test set for this ID
    # ---------------------------------------------------
    test_idx = [i for i, tid in enumerate(test_ids_v) if tid == id]

    if len(test_idx) == 0:
        return None

    test_files_id = [test_files_v[i] for i in test_idx]

    test_feat_list = [
        test_feat_all[f]
        for f in test_files_id
        if f in test_feat_all and test_feat_all[f] is not None
    ]

    if len(test_feat_list) == 0:
        return None

    X_test = np.vstack(test_feat_list)

    if X_test.shape[0] < 2:
        return None

    # ---------------------------------------------------
    # Clip labels from filename
    # ---------------------------------------------------
    y_test_clip = np.array([
        1 if "anomaly" in os.path.basename(f).lower() else 0
        for f in test_files_id
    ])

    # ---------------------------------------------------
    # Frame → clip grouping
    # ---------------------------------------------------
    frame_counts = [M.shape[0] for M in test_feat_list]
    test_groups = np.repeat(
        np.arange(len(frame_counts)), frame_counts
    )

    # ---------------------------------------------------
    # Train autoencoder on NORMAL frames only
    # ---------------------------------------------------
    model = build_model(input_dim)

    model.fit(
        X_train, X_train,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        verbose=verbose
    )

    # ---------------------------------------------------
    # Reconstruction error = anomaly score
    # ---------------------------------------------------
    X_test_hat = model.predict(X_test, batch_size=batch_size, verbose=0)

    frame_scores = np.mean(
        np.square(X_test - X_test_hat),
        axis=1
    )

    # ---------------------------------------------------
    # Aggregate frame scores → clip scores
    # ---------------------------------------------------
    clip_scores = np.array([
        frame_scores[test_groups == g].mean()
        for g in np.unique(test_groups)
    ])

    # ---------------------------------------------------
    # Metrics
    # ---------------------------------------------------
    auc_val = roc_auc_score(y_test_clip, clip_scores)

    fpr, tpr, _ = roc_curve(y_test_clip, clip_scores)
    mask = fpr <= 0.1  # specificity ≥ 0.9
    pauc_val = np.trapezoid(tpr[mask], fpr[mask]) / 0.1

    return {
        "auc": float(auc_val),
        "pauc": float(pauc_val)
    }

In [26]:
id0 = unique_ids[0]
boot_train_files_for_id = [
    f for f, tid in zip(train_files_v, train_ids_v)
    if tid == id0
]
print(len(boot_train_files_for_id))


911


In [27]:
result = run_one_id_if(
    id=id0,
    boot_train_files_for_id=boot_train_files_for_id,
    max_frames_per_clip=500,   # optional
    epochs=5,                  # keep small for testing
    batch_size=256,
    verbose=1
)

print(result)


Epoch 1/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 124.2983
Epoch 2/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 12.3019
Epoch 3/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 11.1441
Epoch 4/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 10.6463
Epoch 5/5
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 10.3357
{'auc': 0.5622358722358722, 'pauc': 0.04447174447174447}


Main bootstrap loop (100 runs total) + save all AUCs

In [28]:
# results_df <- data.frame(...)
results_df = []

# total number of iterations for progress bar
total_iters = R * len(unique_ids)

k = 0

with tqdm(total=total_iters, desc="Bootstrap runs") as pb:
    for r in range(1, R + 1):   # R runs, 1-based like R
        for id in unique_ids:

            # ---------------------------------------------------
            # Select training indices for this ID
            # ---------------------------------------------------
            train_idx = [i for i, tid in enumerate(train_ids_v) if tid == id]

            if len(train_idx) == 0:
                k += 1
                pb.update(1)
                continue

            train_files_id = [train_files_v[i] for i in train_idx]

            # ---------------------------------------------------
            # Bootstrap resampling of training CLIPS
            # ---------------------------------------------------
            boot_train_files_for_id = np.random.choice(
                train_files_id,
                size=len(train_files_id),
                replace=True
            ).tolist()

            # ---------------------------------------------------
            # Run one bootstrap experiment for this ID
            # ---------------------------------------------------
            out = run_one_id_if(
                id=id,
                boot_train_files_for_id=boot_train_files_for_id,
                max_frames_per_clip=None
            )

            if out is not None:
                results_df.append({
                    "run":  r,
                    "id":   id,
                    "auc":  out["auc"],
                    "pauc": out["pauc"]
                })

            k += 1
            pb.update(1)

# ---------------------------------------------------
# Convert to DataFrame
# ---------------------------------------------------
results_df = pd.DataFrame(results_df)

# ---------------------------------------------------
# Save raw results (one row per run + id)
# ---------------------------------------------------
results_df.to_pickle("bootstrap_if_results.pkl")
results_df.to_csv("bootstrap_if_results.csv", index=False)

Bootstrap runs: 100%|██████████| 400/400 [29:14:38<00:00, 263.20s/it]    


In [30]:
results_df = pd.read_pickle("bootstrap_if_results.pkl")
summary_by_id = (
    results_df
    .groupby("id")
    .agg(
        auc_mean=("auc", "mean"),
        auc_sd=("auc", "std"),
        pauc_mean=("pauc", "mean"),
        pauc_sd=("pauc", "std")
    )
    .reset_index()
)

print(summary_by_id)


   id  auc_mean    auc_sd  pauc_mean   pauc_sd
0  00  0.565992  0.012174   0.046715  0.007923
1  02  0.743641  0.027012   0.178844  0.022665
2  04  0.575886  0.018276   0.088161  0.014039
3  06  0.838256  0.024190   0.299640  0.027758


In [31]:
mean_auc_per_run = (
    results_df
    .groupby("run", as_index=False)
    .agg(mean_auc=("auc", "mean"))
)

print("\nMean AUC across IDs per run:")
print(mean_auc_per_run)


Mean AUC across IDs per run:
    run  mean_auc
0     1  0.676597
1     2  0.663341
2     3  0.680135
3     4  0.673567
4     5  0.685850
..  ...       ...
95   96  0.676896
96   97  0.676267
97   98  0.673952
98   99  0.686082
99  100  0.684501

[100 rows x 2 columns]


In [32]:
print("\nOverall mean AUC:", results_df["auc"].mean())
print("Overall mean pAUC:", results_df["pauc"].mean())


Overall mean AUC: 0.680943876956309
Overall mean pAUC: 0.1533399518984887
